In [1]:
import praw
from datetime import datetime
import pandas as pd
import csv
import spacy
from transformers import pipeline
import time
import prawcore.exceptions



c:\Users\Ben\anaconda3\envs\is310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reddit = praw.Reddit(
    client_id="EKbE2y5TtQ2YTCVz39-kJg", 
    client_secret="r6OdI-xsowqL4k2U5gNjJ4odaW5GHw",
    user_agent="IS310 by u/Sufficient-Explorer1"
)

# Confirm connection (should print True)
print("Read-only mode:", reddit.read_only)
""" 
# Test: print top 5 posts from r/technology
for post in reddit.subreddit("politics").hot(limit=5):
    print(f"{post.title} (Score: {post.score})")
    print(f"URL: {post.url}")
    print(f"Post date: {post.created_utc}")
    print(f"Post date (UTC): {datetime.fromtimestamp(post.created_utc)}\n")
 """

Read-only mode: True


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


' \n# Test: print top 5 posts from r/technology\nfor post in reddit.subreddit("politics").hot(limit=5):\n    print(f"{post.title} (Score: {post.score})")\n    print(f"URL: {post.url}")\n    print(f"Post date: {post.created_utc}")\n    print(f"Post date (UTC): {datetime.fromtimestamp(post.created_utc)}\n")\n '

In [3]:

def getReasonablyUniqueIdentifier(post):
     title = post.title
     
     invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*', ' ', '\n', '\r']
     for char in invalid_chars:
        title = title.replace(char, "_")        
     title = title[:20]
     date = datetime.fromtimestamp(post.created_utc)
     date_str = date.strftime("%Y%m%d_%H%M%S")
     return f"{title}---{date_str}"

def createCommentCSV(post, sleep_time=0):
    try:
        # Replace "MoreComments" objects to get all comments
        post.comments.replace_more(limit=None)
        comments = post.comments.list()
        
        # Generate a unique filename for the post
        filename = getReasonablyUniqueIdentifier(post)

        # Write comments to a CSV file
        with open(f"post_comments/{filename}.csv", "w", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Comment", "Author", "Date", "Score"])
            
            for comment in comments:
                writer.writerow([
                    comment.body.replace("\n", " "),  # Replace newlines in the comment body
                    comment.author if comment.author else "[deleted]",  # Handle deleted authors
                    datetime.fromtimestamp(comment.created_utc).isoformat(),  # Format the comment date
                    comment.score  # Comment score
                ])
                # Add a delay to avoid hitting the rate limit
                time.sleep(sleep_time)  # Adjust delay as needed (e.g., 0.5 seconds per comment)
    except Exception as e:
        if sleep_time >= 30:
            raise e
        print(f"Exception encountered. Sleeping for {sleep_time * 10} seconds...")
        time.sleep(sleep_time * 10)  # Wait for the duration specified in the exception
        createCommentCSV(post, sleep_time=sleep_time+0.5)  # Retry after sleeping

In [4]:
def getPosts(subreddit, category, postlimit=100):
    with open(f"post_csvs/posts_{subreddit}_{category}-{datetime.now().date()}.csv", "w", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Title", "URL", "Date", "Score", "Author", "num_comments", "identifier"])
        switcher = {
            "hot": reddit.subreddit(subreddit).hot(limit=postlimit),
            "new": reddit.subreddit(subreddit).new(limit=postlimit),
            "top": reddit.subreddit(subreddit).top(limit=postlimit),
            "controversial": reddit.subreddit(subreddit).controversial(limit=postlimit),
            "rising": reddit.subreddit(subreddit).rising(limit=postlimit),
        }
        time.sleep(2)
        for post in switcher.get(category):
            time.sleep(1)
            try: 
                createCommentCSV(post)
                writer.writerow([
                    post.title.replace("\n", " "),  
                    post.url,
                    datetime.fromtimestamp(post.created_utc).isoformat(),
                    post.score,
                    post.author if post.author else "[deleted]",
                    post.num_comments,
                    getReasonablyUniqueIdentifier(post)
                ])
            except Exception as e:
                print(f"Failed to get post comments for post {post.title}: {e}")
                continue
            

In [5]:
getPosts("politics", "hot", 100)
print("Finished politics hot posts")
getPosts("Conservative", "hot", 100)
print("Finished Conservative hot posts")
getPosts("democrats", "hot", 100)
print("Finished Democrat hot posts")
getPosts("PoliticalDiscussion", "hot", 100)
print("Finished PoliticalDiscussion hot posts")
getPosts("Ask_Politics", "hot", 100)
print("Finished Ask_Politics hot posts")

getPosts("politics", "new", 1000)
print("Finished politics new posts")
getPosts("Conservative", "new", 1000)
print("Finished Conservative new posts")
getPosts("democrats", "new", 1000)
print("Finished Democrat new posts")
getPosts("PoliticalDiscussion", "new", 1000)
print("Finished PoliticalDiscussion new posts")
getPosts("Ask_Politics", "new", 1000)
print("Finished Ask_Politics new posts")

Exception encountered. Sleeping for 0 seconds...
Exception encountered. Sleeping for 5.0 seconds...
Finished politics hot posts
Finished Conservative hot posts
Finished Democrat hot posts
Finished PoliticalDiscussion hot posts
Finished Ask_Politics hot posts
Finished politics new posts
Finished Conservative new posts
Finished Democrat new posts
Finished PoliticalDiscussion new posts
Finished Ask_Politics new posts
